# Using the YOLOv8s Stock Market Future Prediction Model


# Step 5: Define and Test a Simple Trading Strategy


In [ ]:
def simple_moving_average_strategy(df):
    df['SMA20'] = df['close'].rolling(window=20).mean()
    df['SMA50'] = df['close'].rolling(window=50).mean()
    df['signal'] = 0
    df.loc[50:, 'signal'] = np.where(df['SMA20'][50:] > df['SMA50'][50:], 1, 0)
    df['position'] = df['signal'].diff()
    return df

df = simple_moving_average_strategy(prices)


In [15]:
import os
import pandas as pd
import cv2
from ultralyticsplus import YOLO
from datetime import datetime, timedelta

# Load the YOLO model
model_path = 'foduucom/stockmarket-pattern-detection-yolov8'
model = YOLO(model_path)

# Define the base directories for the generated images and the target directory for detected patterns
base_dir = 'EUR_USD/M15'
target_base_dir = 'yolo_pattern/EUR_USD/M15'

# Create the target base directory if it does not exist
os.makedirs(target_base_dir, exist_ok=True)

# DataFrame to store prediction details
predictions_df = pd.DataFrame(columns=[
    'image_path', 'pattern', 'confidence', 'end_date', 'end_price', 'start_date', 'start_price', 'signal', 'tp', 'sl', 'validation', 'profit'
])

def detect_patterns_in_image(image_path):
    results = model.predict(source=image_path)
    return results

def create_target_directory_structure(base_dir, sub_path):
    target_dir = os.path.join(base_dir, sub_path)
    os.makedirs(target_dir, exist_ok=True)
    return target_dir

def save_annotated_image(image_path, results, target_image_path):
    image = cv2.imread(image_path)
    for detection in results[0].boxes:
        x1, y1, x2, y2 = map(int, detection.xyxy[0])
        confidence = detection.conf[0]
        label = detection.cls[0].item()
        pattern = model.names[label]
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, f'{pattern} ({confidence:.2f})', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.imwrite(target_image_path, image)

def get_date_and_price_from_coords(df, x, y):
    num_rows, num_cols = df.shape[0], df.shape[1]
    time_idx = int((x / num_cols) * num_rows)
    price_idx = int((y / num_cols) * num_rows)
    date = df.iloc[time_idx]['time']
    price = df.iloc[price_idx]['close']
    return date, price

def determine_signal(pattern):
    if pattern in ['Head and shoulders top', 'M_Head']:
        return 'sell'
    elif pattern in ['Head and shoulders bottom', 'W_Bottom']:
        return 'buy'
    return 'neutral'

def calculate_tp_sl(signal, price):
    if signal == 'buy':
        tp = price * 1.02
        sl = price * 0.98
    elif signal == 'sell':
        tp = price * 0.98
        sl = price * 1.02
    else:
        tp, sl = None, None
    return tp, sl

def validate_signal(df, start_date, signal, tp, sl):
    filtered_df = df[df['time'] >= start_date]
    for index, row in filtered_df.iterrows():
        price = row['close']
        if signal == 'buy':
            if price >= tp:
                return 1, tp - price
            elif price <= sl:
                return 0, price - sl
        elif signal == 'sell':
            if price <= tp:
                return 1, price - tp
            elif price >= sl:
                return 0, sl - price
    return 0, 0

def process_images_and_save_patterns(base_dir, target_base_dir, predictions_df):
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith('.png'):
                image_path = os.path.join(root, file)
                sub_path = os.path.relpath(root, base_dir)

                # Run the YOLO model on the image
                results = detect_patterns_in_image(image_path)

                # Check if any patterns were detected
                if results[0].boxes:
                    # Create the corresponding target directory
                    target_dir = create_target_directory_structure(target_base_dir, sub_path)
                    
                    # Save the image with detected patterns to the target directory
                    target_image_path = os.path.join(target_dir, file)
                    save_annotated_image(image_path, results, target_image_path)
                    
                    # Extract and save prediction data
                    for detection in results[0].boxes:
                        x1, y1, x2, y2 = map(int, detection.xyxy[0])
                        confidence = detection.conf[0]
                        label = detection.cls[0].item()
                        pattern = model.names[label]
                        
                        # Get start and end dates and prices
                        df = pd.read_csv(f'data/{sub_path.replace("\\", "/")}.csv', parse_dates=['time'])
                        start_date, start_price = get_date_and_price_from_coords(df, x1, y1)
                        end_date, end_price = get_date_and_price_from_coords(df, x2, y2)

                        # Determine signal
                        signal = determine_signal(pattern)

                        # Calculate TP and SL
                        tp, sl = calculate_tp_sl(signal, end_price)

                        # Validate signal and calculate profit/loss
                        validation, profit = validate_signal(df, end_date, signal, tp, sl)

                        # Add data to the DataFrame
                        new_row = pd.DataFrame([{
                            'image_path': target_image_path,
                            'pattern': pattern,
                            'confidence': confidence,
                            'end_date': end_date,
                            'end_price': end_price,
                            'start_date': start_date,
                            'start_price': start_price,
                            'signal': signal,
                            'tp': tp,
                            'sl': sl,
                            'validation': validation,
                            'profit': profit
                        }])
                        predictions_df = pd.concat([predictions_df, new_row], ignore_index=True)

                    print(f'Detected pattern in image: {image_path}, saved to {target_image_path}')

    return predictions_df

# Process the images and save detected patterns and related data
predictions_df = process_images_and_save_patterns(base_dir, target_base_dir, predictions_df)

# Save the predictions DataFrame to a CSV file
predictions_df.to_csv('yolo_pattern/predictions.csv', index=False)



image 1/1 d:\OneDrive\myproject\stockmarket-pattern-detection-yolov8\EUR_USD\M15\2016\2016-01-03_22-00_to_2016-01-18_07-45.png: 480x640 2 Head and shoulders bottoms, 307.6ms
Speed: 5.0ms preprocess, 307.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Detected pattern in image: EUR_USD/M15\2016\2016-01-03_22-00_to_2016-01-18_07-45.png, saved to yolo_pattern/EUR_USD/M15\2016\2016-01-03_22-00_to_2016-01-18_07-45.png

image 1/1 d:\OneDrive\myproject\stockmarket-pattern-detection-yolov8\EUR_USD\M15\2016\2016-01-11_03-00_to_2016-01-25_12-45.png: 480x640 4 Head and shoulders bottoms, 2 Head and shoulders tops, 220.1ms
Speed: 5.0ms preprocess, 220.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Detected pattern in image: EUR_USD/M15\2016\2016-01-11_03-00_to_2016-01-25_12-45.png, saved to yolo_pattern/EUR_USD/M15\2016\2016-01-11_03-00_to_2016-01-25_12-45.png

image 1/1 d:\OneDrive\myproject\stockmarket-pattern-detection-yolov8\EUR_USD\M15\2016\2016-01-

In [9]:
print(results[0])


ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'down', 1: 'up'}
obb: None
orig_img: array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
      